<a href="https://colab.research.google.com/github/CKCritter/Why-Should-I-Trust-You-Group-7/blob/main/Tubespam_Lime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installs  
You only need to run this cell once

In [ ]:
!pip install -U lime
!pip install -U scikit-learn
!pip install -U pandas

Downloading datasets  
You only need to run this cell once

In [3]:
from google.colab import drive 
drive.mount('/content/gdrive')

%cd /content/gdrive/MyDrive
!mkdir team7-project-3
%cd team7-project-3
!ls

Mounted at /content/gdrive
/content/gdrive/MyDrive
mkdir: cannot create directory ‘team7-project-3’: File exists
/content/gdrive/MyDrive/team7-project-3
__MACOSX		 Youtube05-Shakira.csv
Youtube01-Psy.csv	 YouTube-Spam-Collection-v1.zip
Youtube02-KatyPerry.csv  YouTube-Spam-Collection-v1.zip.1
Youtube03-LMFAO.csv	 YouTube-Spam-Collection-v1.zip.2
Youtube04-Eminem.csv	 YouTube-Spam-Collection-v1.zip.3


In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00380/YouTube-Spam-Collection-v1.zip
!unzip YouTube-Spam-Collection-v1.zip

In [4]:
path = "/content/gdrive/My Drive/team7-project-3" #change this path to the folder containing your tubespam dataset

Imports

In [5]:
import lime
import sklearn
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

import pandas

This cell sets up the datasets. By default they are concatonated. If you would like to test only one dataset simply uncomment one of the "data=tx" lines, the most recent will take priority

In [73]:
t1 = pandas.read_csv("Youtube01-Psy.csv")
t2 = pandas.read_csv("Youtube02-KatyPerry.csv")
t3 = pandas.read_csv("Youtube03-LMFAO.csv")
t4 = pandas.read_csv("Youtube04-Eminem.csv")
t5 = pandas.read_csv("Youtube05-Shakira.csv")

#All data concatonated
cv = CountVectorizer()
data = pandas.merge(t1, t2, how="outer")
data = pandas.merge(data, t3, how="outer")
data = pandas.merge(data, t4, how="outer")
data = pandas.merge(data, t5, how="outer")

#one dataset
#data = t1
#data = t2
#data = t3
#data = t4
#data = t5

data_x = cv.fit_transform(data["CONTENT"])
data_y = data["CLASS"]

Bernoulli Naive Bayes Classifier

In [74]:
bnb = BernoulliNB()

train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, test_size=0.1)#This is a hack

bnb.fit(train_x, train_y)

acc = (test_y == bnb.predict(test_x)).sum()
print(acc, len(test_y), acc / len(test_y))

acc = (train_y == bnb.predict(train_x)).sum()
print(acc, len(train_y), acc / len(train_y))

167 196 0.8520408163265306
1605 1760 0.9119318181818182


LIME

In [92]:
#Modified from the lime repository's very good tutorial
from sklearn.pipeline import make_pipeline
from lime.lime_text import LimeTextExplainer

c = make_pipeline(cv, bnb)
class_names = ['ham', 'spam']

explainer = LimeTextExplainer(class_names=class_names)

import random

idx = random.randrange(0, len(data["CONTENT"]))
exp = explainer.explain_instance(data["CONTENT"][idx], c.predict_proba, num_features=6)
print('Document id: %d' % idx)
print('ProbabiliPty(spam) =', c.predict_proba([data["CONTENT"][idx]])[0, 1])
print('ProbabiliPty(ham) =', c.predict_proba([data["CONTENT"][idx]])[0, 0])
print('True class: %s' % class_names[data_y[idx]])
exp.as_list()

Document id: 329
ProbabiliPty(spam) = 0.9999934202544657
ProbabiliPty(ham) = 6.579745536072133e-06
True class: spam


[('com', 0.26969127211352395),
 ('https', 0.23505363587823588),
 ('FOLLOW', 0.1838886702994772),
 ('thanks', 0.17564382380825086),
 ('ON', 0.14080604489606138),
 ('MY', 0.132204204673946)]